# Preprocess Gene Annotations

This notebook creates a table of gene annotations by:
1. Querying Biomart for all Ensembl IDs in the database
2. Querying MyGene for annotation about those IDs
3. Querying Ensembl for the most recent Ensembl release for each ID
4. Building a permalink to the Ensembl archive page for each ID

This gene annotation table is read in by `agoradataprocessing/process.py` to be used in the `gene_info` transformation. 

***Note:*** *This notebook is exploratory and should eventually be converted to a Python script that is run through an automated process.*

## Installation requirements

#### Linux / Windows / Mac

Install R: https://cran.r-project.org/

Install Python and agora-data-tools following the instructions in this repository's README. This notebook assumes it is being run from the same `pipenv` virtual environment as agora-data-tools. 

Then install the following packages using `pip`:
```
pip install rpy2 mygene
```

#### Note for Macs with M1 chips (2020 and newer)

Install as above, but make sure that your R installation is the arm64 version (R-4.X.X-arm64.pkg) so that the architecture matches what pip is using. 

In [1]:
from rpy2.robjects import r
from os import name
import pandas as pd
import mygene
import numpy as np
import requests
import synapseclient
import agoradatatools.etl.transform as transform
import agoradatatools.etl.utils as utils
import agoradatatools.etl.extract as extract

r('if (!require("BiocManager", character.only = TRUE)) { install.packages("BiocManager") }')
r('if (!require("biomaRt")) { BiocManager::install("biomaRt") }')

r.library('biomaRt')

biomart_filename = '../output/biomart_ensg_list.txt'
archive_filename = '../output/ensembl_archive_list.csv'
config_filename = '../../config.yaml'



  is available with R version '4.2'; see https://bioconductor.org/install




# Part 1: Get gene annotation data

## Query Biomart for a list of all Ensembl IDs in the database of human genes. 

Here we use the R library `biomaRt`. There is no canonical Python library with the features we need for this script. 

In [2]:
# Sometimes Biomart doesn't respond and the command needs to be sent again. Try up to 5 times.
for T in range(5):
    try:
        mart = r.useEnsembl(biomart = 'genes', dataset = 'hsapiens_gene_ensembl')
        ensembl_ids = r.getBM(attributes = 'ensembl_gene_id', mart = mart, useCache = False)
        
    except rpy2.rinterface.RRuntimeError as err:
        print(err)
        print('Trying again...')
    
    else: 
        break

if ensembl_ids.nrow == 0:
    print('Biomart was unresponsive after 5 attempts. Try again later.')

else:
    # Convert the ensembl_gene_id column from R object to a python list
    ensembl_ids = list(ensembl_ids.rx2('ensembl_gene_id'))

    print(ensembl_ids[0:5])
    print(str(len(ensembl_ids)) + " genes found.")

['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460']
69299 genes found.


## Add Ensembl IDs from data sets that will be processed by agora-data-tools

Some of these datasets have older/retired Ensembl IDs that no longer exist in the current Ensembl database. Loop through all data sets in the config file to get all Ensembl IDs they use, and add missing ones to the gene list. 

In [3]:
config = utils._get_config(config_path = config_filename)
datasets = utils._find_config_by_name(config, 'datasets')

files = {}

for dataset in datasets:
    dataset_name = list(dataset.keys())[0]
    
    for entity in dataset[dataset_name]['files']:
        entity_id = entity['id']
        entity_format = entity['format']
        entity_name = entity['name']
        
        # Ignore json files, which are post-processed and not what we're interested in. 
        # Also ignore "gene_metadata" since that's the file we're making here.
        if entity_format != 'json' and entity_name != "gene_metadata":
            files[entity_name] = (entity_id, entity_format)

# There are some duplicate synID's in this list but that doesn't really matter
files

{'genes_biodomains': ('syn44151254.1', 'csv'),
 'neuropath_regression_results': ('syn22017882.5', 'csv'),
 'agora_proteomics': ('syn18689335.3', 'csv'),
 'agora_proteomics_tmt': ('syn35221005.2', 'csv'),
 'target_exp_validation_harmonized': ('syn24184512.6', 'csv'),
 'metabolomics': ('syn26064497.1', 'feather'),
 'igap': ('syn12514826.4', 'csv'),
 'eqtl': ('syn12514912.3', 'csv'),
 'proteomics': ('syn18689335.3', 'csv'),
 'rna_expression_change': ('syn27211942.1', 'tsv'),
 'target_list': ('syn12540368.36', 'csv'),
 'median_expression': ('syn27211878.2', 'csv'),
 'druggability': ('syn13363443.11', 'csv'),
 'team_info': ('syn12615624.13', 'csv'),
 'team_member_info': ('syn12615633.15', 'csv'),
 'overall_scores': ('syn25575156.13', 'table'),
 'networks': ('syn11685347.1', 'csv'),
 'diff_exp_data': ('syn27211942.1', 'tsv'),
 'proteomics_tmt': ('syn35221005.2', 'csv')}

### We should now have a list of all raw data files ingested. Get each one and add its Ensembl IDs to the gene list.

In [4]:
syn = utils._login_to_synapse(token = None) # Assumes you have already logged in with a valid token

# The various column names used to store Ensembl IDs in the files
col_names = ['ENSG', 'ensembl_gene_id', 'GeneID']

for file in files.keys():
    df = extract.get_entity_as_df(syn_id=files[file][0],
                                  source=files[file][1],
                                  syn=syn)
    file_ensembl_ids = None
    
    for C in col_names:
        if C in df.columns:
            file_ensembl_ids = df[C]
    
    # networks file is a special case
    if file == "networks":
        file_ensembl_ids = pd.melt(df[['geneA_ensembl_gene_id','geneB_ensembl_gene_id']])['value']
        
    if file_ensembl_ids is not None:
        ensembl_ids = ensembl_ids + file_ensembl_ids.tolist()
        if "n/A" in file_ensembl_ids.tolist():
            print(file + " has an n/A Ensembl ID")
        if np.NaN in file_ensembl_ids.tolist():
            print(file + " has an NaN Ensembl ID")

Welcome, Jaclyn Beck!



INFO:synapseclient_default:Welcome, Jaclyn Beck!



target_exp_validation_harmonized has an n/A Ensembl ID


In [5]:
ensembl_ids = list(set(ensembl_ids))
ensembl_ids.remove("n/A") # Necessary because one data set has an Ensembl ID set to "n/A"

# Convert to a pandas data frame
ensembl_ids_df = pd.DataFrame({'ensembl_gene_id': ensembl_ids})
len(ensembl_ids_df)

70446

In [6]:
# Write to a file
ensembl_ids_df.to_csv(path_or_buf = biomart_filename, sep = '\t', header = False, index = False)

## Get info on each gene from mygene

In [7]:
mg = mygene.MyGeneInfo()

mygene_output = mg.getgenes(ensembl_ids_df['ensembl_gene_id'], 
                            fields=["symbol", "name", "summary", "type_of_gene", "alias"], 
                            as_dataframe=True)

mygene_output.index.rename("ensembl_gene_id", inplace=True)
mygene_output.head()

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:querying 1001-2000...
INFO:biothings.client:done.
INFO:biothings.client:querying 2001-3000...
INFO:biothings.client:done.
INFO:biothings.client:querying 3001-4000...
INFO:biothings.client:done.
INFO:biothings.client:querying 4001-5000...
INFO:biothings.client:done.
INFO:biothings.client:querying 5001-6000...
INFO:biothings.client:done.
INFO:biothings.client:querying 6001-7000...
INFO:biothings.client:done.
INFO:biothings.client:querying 7001-8000...
INFO:biothings.client:done.
INFO:biothings.client:querying 8001-9000...
INFO:biothings.client:done.
INFO:biothings.client:querying 9001-10000...
INFO:biothings.client:done.
INFO:biothings.client:querying 10001-11000...
INFO:biothings.client:done.
INFO:biothings.client:querying 11001-12000...
INFO:biothings.client:done.
INFO:biothings.client:querying 12001-13000...
INFO:biothings.client:done.
INFO:biothings.client:querying 13001-14000...
INFO:biothings

,_id,_version,name,symbol,type_of_gene,alias,summary,notfound
ensembl_gene_id,,,,,,,,
ENSG00000234956,100507406,1.0,long intergenic non-protein coding RNA 2539,LINC02539,ncRNA,NaN,NaN,NaN
ENSG00000260584,ENSG00000260584,1.0,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000100376,55007,3.0,family with sequence similarity 118 member A,FAM118A,protein-coding,C22orf8,Enables identical protein binding activity. Pr...,NaN
ENSG00000215458,284837,2.0,apoptosis associated transcript in bladder cancer,AATBC,ncRNA,NaN,NaN,NaN
ENSG00000236028,ENSG00000236028,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print("Annotations found for " + str(sum(mygene_output['notfound'].isna())) + " genes.")
print("No annotations found for " + str(sum(mygene_output['notfound'] == True)) + " genes.")

Annotations found for 70106 genes.
No annotations found for 1152 genes.


# Part 2: Clean the data

## Join and standardize columns / values

For consistency with the `agora-data-tools` transform process, this uses the etl standardize functions.

In [9]:
# This merge may not be strictly necessary? mygene should return at least one row all genes queried even if  
# it can't find the gene in the database
gene_table_merged = pd.merge(left = ensembl_ids_df, right = mygene_output, how = 'left', on = 'ensembl_gene_id')

gene_table_merged = transform.standardize_column_names(gene_table_merged)
gene_table_merged = transform.standardize_values(gene_table_merged)

print(gene_table_merged.shape)
gene_table_merged.head()

(71258, 9)


,ensembl_gene_id,_id,_version,name,symbol,type_of_gene,alias,summary,notfound
0,ENSG00000234956,100507406,1.0,long intergenic non-protein coding RNA 2539,LINC02539,ncRNA,NaN,NaN,NaN
1,ENSG00000260584,ENSG00000260584,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSG00000100376,55007,3.0,family with sequence similarity 118 member A,FAM118A,protein-coding,C22orf8,Enables identical protein binding activity. Pr...,NaN
3,ENSG00000215458,284837,2.0,apoptosis associated transcript in bladder cancer,AATBC,ncRNA,NaN,NaN,NaN
4,ENSG00000236028,ENSG00000236028,1.0,NaN,NaN,NaN,NaN,NaN,NaN


## Fix alias field

Fix `NaN` values in the `alias` field and make sure every alias value is a list, not a string.

In [10]:
# NaN or NULL alias values become empty lists
for row in gene_table_merged.loc[gene_table_merged['alias'].isnull(), 'alias'].index:
    gene_table_merged.at[row, 'alias'] = []

# Some alias values are a single string, not a list. Turn them into lists here.
gene_table_merged['alias'] = gene_table_merged['alias'].apply(lambda cell: cell if isinstance(cell, list) else [cell])

## Remove duplicate Ensembl IDs from the list. 

Duplicates in the list typically have the same Ensembl ID but different gene symbols. This usually happens when a single Ensembl ID maps to multiple Entrez IDs in the NCBI database. There's not a good way to reconcile this, so just use the first entry in the list for each ensembl ID and discard the rest, which is what the Agora front end does. The gene symbols of duplicate rows are then added as aliases to the matching unique row.

In [11]:
# duplicated() will return true if the ID is a duplicate and is not the first one to appear the list. 
dupes = gene_table_merged['ensembl_gene_id'].duplicated()
dupe_vals = gene_table_merged[dupes]

# Rows with duplicated Ensembl IDs
gene_table_merged.loc[gene_table_merged['ensembl_gene_id'].isin(dupe_vals['ensembl_gene_id'])]

,ensembl_gene_id,_id,_version,name,symbol,type_of_gene,alias,summary,notfound
127,ENSG00000284156,124905408,2.0,double homeobox protein 4,LOC124905408,protein-coding,[],NaN,NaN
128,ENSG00000284156,124905411,2.0,double homeobox protein 4,LOC124905411,protein-coding,[],NaN,NaN
129,ENSG00000284156,124906452,2.0,double homeobox protein 4,LOC124906452,protein-coding,[],NaN,NaN
130,ENSG00000284156,124906454,2.0,double homeobox protein 4,LOC124906454,protein-coding,[],NaN,NaN
131,ENSG00000284156,124906457,2.0,double homeobox protein 4,LOC124906457,protein-coding,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...
57793,ENSG00000273730,124908522,1.0,5.8S ribosomal RNA,LOC124908522,rRNA,[],NaN,NaN
63658,ENSG00000249738,105377683,1.0,uncharacterized LOC105377683,LOC105377683,ncRNA,[],NaN,NaN
63659,ENSG00000249738,285626,1.0,uncharacterized LOC285626,LOC285626,ncRNA,[],NaN,NaN
70770,ENSG00000260788,105371366,1.0,uncharacterized LOC105371366,LOC105371366,ncRNA,[],NaN,NaN


In [12]:
# Remove duplicates from the list
gene_table_merged = gene_table_merged[dupes == False].reset_index()

# For each duplicate row, add its symbol as an alias
for row in dupe_vals.index:
    match = gene_table_merged['ensembl_gene_id'] == dupe_vals['ensembl_gene_id'][row]
    match_ind = gene_table_merged[match].index[0] # There should only be one row

    # Add the duplicate's symbol to the alias list
    gene_table_merged.at[match_ind, 'alias'].append(dupe_vals['symbol'][row])
    
    # Make sure we didn't add duplicate aliases
    gene_table_merged.at[match_ind, 'alias'] = list(set(gene_table_merged.at[match_ind, 'alias']))

print(gene_table_merged.shape)

# Printed out table should have unique Ensembl IDs with aliases properly added
gene_table_merged.loc[gene_table_merged['ensembl_gene_id'].isin(dupe_vals['ensembl_gene_id'])]

(70446, 10)


,index,ensembl_gene_id,_id,_version,name,symbol,type_of_gene,alias,summary,notfound
127,127,ENSG00000284156,124905408,2.0,double homeobox protein 4,LOC124905408,protein-coding,"[LOC124905409, LOC124906466, LOC124906454, LOC...",NaN,NaN
1343,1360,ENSG00000283878,124905408,2.0,double homeobox protein 4,LOC124905408,protein-coding,"[LOC124906458, LOC124906460, LOC124905409, LOC...",NaN,NaN
2039,2073,ENSG00000268674,124905699,1.0,FAM231A/C-like protein LOC102723383,LOC124905699,protein-coding,"[LOC124905693, LOC124905556, LOC124903857]",NaN,NaN
4943,4980,ENSG00000278747,124904706,1.0,U6 spliceosomal RNA,LOC124904706,snRNA,"[LOC124906683, LOC124904108]",NaN,NaN
8611,8650,ENSG00000273624,124905327,1.0,U6 spliceosomal RNA,LOC124905327,snRNA,"[LOC124905519, LOC124900632]",NaN,NaN
9582,9623,ENSG00000282767,124900566,1.0,U8 small nucleolar RNA,LOC124900566,snoRNA,[LOC124900359],NaN,NaN
15070,15112,ENSG00000284502,124905408,2.0,double homeobox protein 4,LOC124905408,protein-coding,"[LOC124905409, LOC124906466, LOC124906454, LOC...",NaN,NaN
15558,15617,ENSG00000280619,107987420,1.0,uncharacterized LOC107987420,LOC107987420,ncRNA,[LOC105374682],NaN,NaN
16366,16426,ENSG00000283898,124905408,2.0,double homeobox protein 4,LOC124905408,protein-coding,"[LOC124906458, LOC124906460, LOC124905409, LOC...",NaN,NaN
17629,17706,ENSG00000288164,102723573,2.0,40S ribosomal protein S8-like,LOC102723573,protein-coding,[LOC102724737],NaN,NaN


# Part 3: Create Ensembl archive permalinks

## Get a table of Ensembl archive URLs

This is where we need to use the R biomaRt library specifically, instead of any of the available Python interfaces to Biomart, to get a table of Ensembl release versions and their corresponding archive URLs. 

In [13]:
archive_df = r.listEnsemblArchives()
archive_df.to_csvfile(path = archive_filename, row_names = False, quote = False)

print(archive_df)

             name     date                                 url version
1  Ensembl GRCh37 Feb 2014          https://grch37.ensembl.org  GRCh37
2     Ensembl 109 Feb 2023 https://feb2023.archive.ensembl.org     109
3     Ensembl 108 Oct 2022 https://oct2022.archive.ensembl.org     108
4     Ensembl 107 Jul 2022 https://jul2022.archive.ensembl.org     107
5     Ensembl 106 Apr 2022 https://apr2022.archive.ensembl.org     106
6     Ensembl 105 Dec 2021 https://dec2021.archive.ensembl.org     105
7     Ensembl 104 May 2021 https://may2021.archive.ensembl.org     104
8     Ensembl 103 Feb 2021 https://feb2021.archive.ensembl.org     103
9     Ensembl 102 Nov 2020 https://nov2020.archive.ensembl.org     102
10    Ensembl 101 Aug 2020 https://aug2020.archive.ensembl.org     101
11    Ensembl 100 Apr 2020 https://apr2020.archive.ensembl.org     100
12     Ensembl 99 Jan 2020 https://jan2020.archive.ensembl.org      99
13     Ensembl 98 Sep 2019 https://sep2019.archive.ensembl.org      98
14    

## Query Ensembl for each gene's version

Ensembl's REST API can only take 1000 genes at once, so this is looped to query groups of 1000. 

In [14]:
url = "https://rest.ensembl.org/archive/id"
headers = {"Content-Type" : "application/json", "Accept" : "application/json"}

ids = gene_table_merged['ensembl_gene_id'].tolist()
print(len(ids))

# We can only query 1000 genes at a time
batch_ind = range(0, len(ids), 1000)
results = []

for B in batch_ind:
    end = min(len(ids), B + 1000)
    print("Querying genes " + str(B+1) + " - " + str(end))
    
    request_data = '{ "id" : ' + str(ids[B:end]) + ' }'
    request_data = request_data.replace("'", "\"")
    
    ok = False
    tries = 0
    
    while tries < 5 and not ok:
        try:
            res = requests.post(url, headers=headers, data=request_data)
            ok = res.ok
        except:
            ok = False
        
        tries = tries + 1
        
        if not ok:
            #res.raise_for_status()
            print("Error retrieving Ensembl versions for genes " + str(B+1) + " - " + str(end) + 
                  ". Trying again...")
        else:
            results = results + res.json()
            break

print(len(results))

versions = pd.json_normalize(results)

versions.tail()

70446
Querying genes 1 - 1000
Querying genes 1001 - 2000
Querying genes 2001 - 3000
Querying genes 3001 - 4000
Querying genes 4001 - 5000
Querying genes 5001 - 6000
Querying genes 6001 - 7000
Querying genes 7001 - 8000
Querying genes 8001 - 9000
Querying genes 9001 - 10000
Querying genes 10001 - 11000
Querying genes 11001 - 12000
Querying genes 12001 - 13000
Querying genes 13001 - 14000
Querying genes 14001 - 15000
Querying genes 15001 - 16000
Querying genes 16001 - 17000
Querying genes 17001 - 18000
Querying genes 18001 - 19000
Querying genes 19001 - 20000
Querying genes 20001 - 21000
Querying genes 21001 - 22000
Querying genes 22001 - 23000
Querying genes 23001 - 24000
Querying genes 24001 - 25000
Querying genes 25001 - 26000
Querying genes 26001 - 27000
Error retrieving Ensembl versions for genes 26001 - 27000. Trying again...
Error retrieving Ensembl versions for genes 26001 - 27000. Trying again...
Querying genes 27001 - 28000
Querying genes 28001 - 29000
Querying genes 29001 - 30

,assembly,version,peptide,latest,type,id,is_current,release,possible_replacement
70441,GRCh38,2,None,ENSG00000233405.2,Gene,ENSG00000233405,1,109,[]
70442,GRCh38,1,None,ENSG00000284485.1,Gene,ENSG00000284485,1,109,[]
70443,GRCh38,1,None,ENSG00000234640.1,Gene,ENSG00000234640,1,109,[]
70444,GRCh38,11,None,ENSG00000188596.11,Gene,ENSG00000188596,1,109,[]
70445,GRCh38,3,None,ENSG00000251205.3,Gene,ENSG00000251205,1,109,[]


In [15]:
versions.groupby('release').size()

release
100       23
101        8
102       16
103       15
104       19
105        9
106       32
107       10
108        4
109    69299
80        21
81         2
82        10
84       673
87        61
89        20
91        75
93        53
95        33
96        31
97        18
98         8
99         6
dtype: int64

In [16]:
# Check that all IDs are the same between the result and the gene table
print(len(versions['id']))
print(len(gene_table_merged))
print(all(versions['id'].isin(gene_table_merged['ensembl_gene_id'])) and 
      all(gene_table_merged['ensembl_gene_id'].isin(versions['id'])))

70446
70446
True


In [17]:
# Make sure everything is GRCh38, not GRCh37
all(versions['assembly'] == "GRCh38")

True

## Create permalinks based on archive version

**Not all of these versions have an archive.** We can go back to the closest previous archive for these but the link isn't guaranteed to work.

In [18]:
archive_table = pd.read_csv(archive_filename)

# Remove GRCh37 from the archive list
archive_table = archive_table[archive_table['version'] != "GRCh37"].reset_index()

archive_table['numeric_version'] = archive_table['version'].astype(int)

def closest_release(release, archive_table):
    if release in archive_table:
        return release
    
    return max([V for V in archive_table['numeric_version'] if V <= release])

In [19]:
versions['closest_release'] = 0

releases = versions['release'].drop_duplicates().astype(int)

# Only have to call closest_release once per version, instead of >70k times
for release in releases:
    versions.loc[versions['release'] == str(release), 'closest_release'] = closest_release(release, archive_table)
    
versions.groupby('closest_release').size()

closest_release
80       862
93        53
95        33
96        31
97        18
98         8
99         6
100       23
101        8
102       16
103       15
104       19
105        9
106       32
107       10
108        4
109    69299
dtype: int64

In [20]:
versions['permalink'] = ''

for i in versions.index:
    match = archive_table['numeric_version'] == versions.at[i, 'closest_release']
    url = archive_table.loc[match, 'url'].to_string(index = False)
    if len(url) > 0:
        versions.at[i, 'permalink'] = url + "/Homo_sapiens/Gene/Summary?db=core;g=" + versions.at[i, 'id']

versions.head()

,assembly,version,peptide,latest,type,id,is_current,release,possible_replacement,closest_release,permalink
0,GRCh38,7,None,ENSG00000234956.7,Gene,ENSG00000234956,1,109,[],109,https://feb2023.archive.ensembl.org/Homo_sapie...
1,GRCh38,1,None,ENSG00000260584.1,Gene,ENSG00000260584,1,109,[],109,https://feb2023.archive.ensembl.org/Homo_sapie...
2,GRCh38,12,None,ENSG00000100376.12,Gene,ENSG00000100376,1,109,[],109,https://feb2023.archive.ensembl.org/Homo_sapie...
3,GRCh38,8,None,ENSG00000215458.8,Gene,ENSG00000215458,1,109,[],109,https://feb2023.archive.ensembl.org/Homo_sapie...
4,GRCh38,1,None,ENSG00000236028.1,Gene,ENSG00000236028,1,109,[],109,https://feb2023.archive.ensembl.org/Homo_sapie...


In [21]:
versions[versions['closest_release'] < 100].head()

,assembly,version,peptide,latest,type,id,is_current,release,possible_replacement,closest_release,permalink
225,GRCh38,1,None,ENSG00000274983.1,Gene,ENSG00000274983,,84,[],80,https://may2015.archive.ensembl.org/Homo_sapie...
389,GRCh38,2,None,ENSG00000274155.2,Gene,ENSG00000274155,,84,[],80,https://may2015.archive.ensembl.org/Homo_sapie...
424,GRCh38,1,None,ENSG00000207765.1,Gene,ENSG00000207765,,84,[],80,https://may2015.archive.ensembl.org/Homo_sapie...
569,GRCh38,1,None,ENSG00000276117.1,Gene,ENSG00000276117,,84,[],80,https://may2015.archive.ensembl.org/Homo_sapie...
818,GRCh38,1,None,ENSG00000278423.1,Gene,ENSG00000278423,,84,[],80,https://may2015.archive.ensembl.org/Homo_sapie...


In [22]:
print(versions['permalink'][0])
print(versions['permalink'][25])

https://feb2023.archive.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000234956
https://feb2023.archive.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000257530


In [23]:
# Does every gene have an associated URL?
url_base_len = len(archive_table['url'][0]) + 1
all([len(url) > url_base_len for url in versions['permalink']])

True

# Part 4: Add permalinks to the gene table

In [24]:
versions = versions[['id', 'release', 'possible_replacement', 'permalink']]
versions.rename(columns={'id': 'ensembl_gene_id', 'release': 'ensembl_release'}, inplace=True)

gene_table_merged = pd.merge(left = gene_table_merged, right = versions, how = 'left', on = 'ensembl_gene_id')

print(gene_table_merged.shape)
gene_table_merged.head()

(70446, 13)


,index,ensembl_gene_id,_id,_version,name,symbol,type_of_gene,alias,summary,notfound,ensembl_release,possible_replacement,permalink
0,0,ENSG00000234956,100507406,1.0,long intergenic non-protein coding RNA 2539,LINC02539,ncRNA,[],NaN,NaN,109,[],https://feb2023.archive.ensembl.org/Homo_sapie...
1,1,ENSG00000260584,ENSG00000260584,1.0,NaN,NaN,NaN,[],NaN,NaN,109,[],https://feb2023.archive.ensembl.org/Homo_sapie...
2,2,ENSG00000100376,55007,3.0,family with sequence similarity 118 member A,FAM118A,protein-coding,[C22orf8],Enables identical protein binding activity. Pr...,NaN,109,[],https://feb2023.archive.ensembl.org/Homo_sapie...
3,3,ENSG00000215458,284837,2.0,apoptosis associated transcript in bladder cancer,AATBC,ncRNA,[],NaN,NaN,109,[],https://feb2023.archive.ensembl.org/Homo_sapie...
4,4,ENSG00000236028,ENSG00000236028,1.0,NaN,NaN,NaN,[],NaN,NaN,109,[],https://feb2023.archive.ensembl.org/Homo_sapie...


### Final cleanup
Unfilled "possible_replacement" entries should be changed from NaN to empty lists. 

"possible_replacement" entries that have data in them exist as a list of dicts, and need to have the Ensembl IDs pulled out of them as a list of strings. 

Remove unneeded columns. 

In [25]:
for row in gene_table_merged.loc[gene_table_merged['possible_replacement'].isnull(), 'possible_replacement'].index:
    gene_table_merged.at[row, 'possible_replacement'] = []
    
gene_table_merged['possible_replacement'] = gene_table_merged.apply(
    lambda row: row['possible_replacement'] if len(row['possible_replacement']) == 0
        else [x['stable_id'] for x in row['possible_replacement']],
        axis=1
)
    
gene_table_merged.drop(columns=['index', '_id', '_version', 'notfound'], inplace=True)

### Write to a file
This will get uploaded to Synapse as [syn25953363](https://www.synapse.org/#!Synapse:syn25953363).

In [26]:
gene_table_merged.to_feather('../output/gene_table_merged_GRCh38.p13.feather')